# Cost Analysis

## 1. Import Library

In [1]:
import pandas as pd
import numpy as np
import pyodbc # SQL Connection
import sqlCredentials as sql

## 2. Connect to Databases

In [2]:
#PROCUREMENTDB

proc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

#BookXCenterProduction
prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

## 3. Define Clean up tool

In [3]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

## 4. Define Tables

In [4]:
# Dictionary
pub_dict = """
SELECT *
FROM dbo.PublisherDictionary
"""
# LP All currencies
all_lp = """
SELECT
	[Isbn]
	,[Currency]
	,[Price]
FROM
	[Isbn].[ListPrice]
"""
# Bibliography
bilblo_sql = """
SELECT Isbn
    , Title
    , Publisher
    , Author
FROM 
    Isbn.Bibliography
"""
# Echange Rate
exchange_rate = """
SELECT 
    name,
    rate
FROM dbo.XChange
"""
#Supplier Mega List
megalist_sql = """
SELECT 
    [ISBN]
    ,[Supplier]
    ,[Publisher]
    ,[Currency]
    ,[ListPrice]
    ,[Discount]
    ,[CostUnitPrice]
    ,[MaxQtyPerOrder]
    ,[ShipmentOrigin]
    ,[UnitShippingCost]
    ,[SeaFreightCost] AS sea_shipping_cost
  FROM 
    [Process].[SupplierMegaList]
"""

#Import supplier Excel
#Import short_discount
#Import restrictions

### 4.1 Import Dictionary and clean it

In [5]:
pub_dict = pd.read_sql(pub_dict, proc_db)
pub_dict.head()

,PublisherLong,PublisherShort
0,Glencoe McGraw Hill,MCGRAW
1,Glencoe McGraw-Hill,MCGRAW
2,McGraw Create,MCGRAW
3,MCGRAW HE,MCGRAW HE
4,MCGRAW HE - ISE,MCGRAW HE - ISE


In [6]:
pub_dict.columns = map(str.lower, pub_dict.columns)
pub_dict = pub_dict.fillna('N/A')
for col in list(pub_dict.columns):
    pub_dict[col] = pub_dict.apply(lambda x: clean_up(x[col]), axis =1)

pub_dict.head()

,publisherlong,publishershort
0,GLENCOE MCGRAW HILL,MCGRAW
1,GLENCOE MCGRAW-HILL,MCGRAW
2,MCGRAW CREATE,MCGRAW
3,MCGRAW HE,MCGRAW HE
4,MCGRAW HE - ISE,MCGRAW HE - ISE


### 4.2 Import Bibliography and clean it

In [7]:
# IMport and read biblio
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

,Isbn,Title,Publisher,Author
0,9780000000057,"Text, Music, Context: A Resource Item #: S05...",CPH,LCMS
1,9780000001030,Surgical Procedures (2016),CIMC,MAVCC
2,9780000001061,Anatomy & Physiology Student Edition,CIMC,CIMC
3,9780000001146,Membean Student License,Archer,None
4,9780000002402,None,LWW,None


In [8]:
biblio.columns = map(str.lower, biblio.columns)
biblio = biblio.fillna('N/A')
biblio['isbn'] = biblio['isbn'].astype(str)
for col in list(biblio.columns):
    biblio[col] = biblio.apply(lambda x: clean_up(x[col]), axis =1)
biblio.head()

,isbn,title,publisher,author
0,9780000000057,"TEXT, MUSIC, CONTEXT: A RESOURCE ITEM #: S05...",CPH,LCMS
1,9780000001030,SURGICAL PROCEDURES (2016),CIMC,MAVCC
2,9780000001061,ANATOMY & PHYSIOLOGY STUDENT EDITION,CIMC,CIMC
3,9780000001146,MEMBEAN STUDENT LICENSE,ARCHER,N/A
4,9780000002402,N/A,LWW,N/A


Now we Merge bibliography with the dictionary

In [9]:
biblio2 = pd.merge(biblio, pub_dict, how='left', left_on = 'publisher', right_on = 'publisherlong' )
biblio2.drop(columns = ['publisherlong'], inplace = True)
biblio2.rename(columns={'publishershort': 'pub'}, inplace = True)
biblio2.head()

,isbn,title,publisher,author,pub
0,9780000000057,"TEXT, MUSIC, CONTEXT: A RESOURCE ITEM #: S05...",CPH,LCMS,NaN
1,9780000001030,SURGICAL PROCEDURES (2016),CIMC,MAVCC,NaN
2,9780000001061,ANATOMY & PHYSIOLOGY STUDENT EDITION,CIMC,CIMC,NaN
3,9780000001146,MEMBEAN STUDENT LICENSE,ARCHER,N/A,NaN
4,9780000002402,N/A,LWW,N/A,LWW


### 4.3 Import SupplierMegaList and clean it

In [10]:
megalist = pd.read_sql(megalist_sql,proc_db)
megalist.head()

,ISBN,Supplier,Publisher,Currency,ListPrice,Discount,CostUnitPrice,MaxQtyPerOrder,ShipmentOrigin,UnitShippingCost,sea_shipping_cost
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,1000,UK,3.47,1.0
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,1000,UK,3.47,1.0
2,9781861345806,Zookal,THAMES & HUDSON,AUD,0.00,0.00,28.00,150,AUS,3.47,1.0
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,1000,UK,3.47,1.0
4,9781861345837,Zookal,THAMES & HUDSON,AUD,0.00,0.00,35.00,150,AUS,3.47,1.0


In [11]:
megalist.columns = map(str.lower, megalist.columns)
megalist = megalist.fillna('N/A')
megalist['isbn'] = megalist['isbn'].astype(str)
for col in ['isbn', 'supplier', 'publisher', 'currency', 'shipmentorigin']:
    megalist[col] = megalist.apply(lambda x: clean_up(x[col]), axis =1)
megalist.head()

,isbn,supplier,publisher,currency,listprice,discount,costunitprice,maxqtyperorder,shipmentorigin,unitshippingcost,sea_shipping_cost
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,1000,UK,3.47,1.0
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,1000,UK,3.47,1.0
2,9781861345806,ZOOKAL,THAMES & HUDSON,AUD,0.00,0.00,28.00,150,AUS,3.47,1.0
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,1000,UK,3.47,1.0
4,9781861345837,ZOOKAL,THAMES & HUDSON,AUD,0.00,0.00,35.00,150,AUS,3.47,1.0


In [12]:
megalist_pub = pd.merge(megalist, biblio2, how='left', on = 'isbn' )
megalist_pub.head()

,isbn,supplier,publisher_x,currency,listprice,discount,costunitprice,maxqtyperorder,shipmentorigin,unitshippingcost,sea_shipping_cost,title,publisher_y,author,pub
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,1000,UK,3.47,1.0,N/A,UNIVERSITY PRESS,N/A,UNIVERSITY PRESS
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,1000,UK,3.47,1.0,N/A,N/A,N/A,N/A
2,9781861345806,ZOOKAL,THAMES & HUDSON,AUD,0.00,0.00,28.00,150,AUS,3.47,1.0,N/A,N/A,N/A,N/A
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,1000,UK,3.47,1.0,N/A,UNIVERSITY PRESS,N/A,UNIVERSITY PRESS
4,9781861345837,ZOOKAL,THAMES & HUDSON,AUD,0.00,0.00,35.00,150,AUS,3.47,1.0,N/A,N/A,N/A,N/A


In [13]:
#cleaning pub name
megalist_pub['pub'] = np.where(
    megalist_pub['pub'] == 'N/A'
    , megalist_pub['publisher_x']
    , megalist_pub['pub'])
    
# centralizing price in one column
megalist_pub['price'] = np.where(megalist_pub['listprice'] == 0.00, megalist_pub['costunitprice'], megalist_pub['listprice'])

#clean column names
megalist_pub.rename(columns={'unitshippingcost': 'shipping_cost', 'maxqtyperorder': 'max_qty', 'shipmentorigin': 'origin'}, inplace = True)

#Arrange columns and drop unnecessary columns

megalist_pub = megalist_pub[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost', 'max_qty', 'origin']]

megalist_pub.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1.0,1000,UK
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1.0,1000,UK
2,9781861345806,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,28.00,0.00,3.47,1.0,150,AUS
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1.0,1000,UK
4,9781861345837,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,35.00,0.00,3.47,1.0,150,AUS


### 4.3 Import all_lp and clean it

In [14]:
all_lp = pd.read_sql(all_lp, prod_db)
all_lp.head()

,Isbn,Currency,Price
0,9780061699948,USD,17.99
1,9780061700071,USD,15.99
2,9780061700194,USD,16.99
3,9780061700927,USD,16.99
4,9780061701269,USD,9.99


In [15]:
all_lp.columns = map(str.lower, all_lp.columns)
all_lp = all_lp.fillna('N/A')
all_lp['isbn'] = all_lp['isbn'].astype(str)
for col in ['isbn','currency']:
    all_lp[col] = all_lp.apply(lambda x: clean_up(x[col]), axis =1)

all_lp.head()

,isbn,currency,price
0,9780061699948,USD,17.99
1,9780061700071,USD,15.99
2,9780061700194,USD,16.99
3,9780061700927,USD,16.99
4,9780061701269,USD,9.99


#### 4.3.1 Get biblio with LP

In [16]:
biblio_lp = pd.merge(all_lp, biblio2, how='left', on = 'isbn' )
biblio_lp.drop(columns = ['pub'], inplace = True)
biblio_lp.head()

,isbn,currency,price,title,publisher,author
0,9780061699948,USD,17.99,N/A,HARPER COLLINS,N/A
1,9780061700071,USD,15.99,N/A,HARPER COLLINS,N/A
2,9780061700194,USD,16.99,N/A,HARPER COLLINS,N/A
3,9780061700927,USD,16.99,N/A,HARPER COLLINS,N/A
4,9780061701269,USD,9.99,N/A,HARPER COLLINS,N/A


### 4.4 Import Sup_by_pub and clean it

In [17]:
sup_pub = pd.read_excel('../../../../Vendors/.  Vendors Details\supplier_procurement_details.xlsx',  sheet_name='pub')
sup_pub.head()

,Supplier,Publisher,Currency,Discount,Shipping Cost,max quantity,Min,ShipmentOrigin,Shipping Cost Sea
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUS,1.0
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUS,1.0
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUS,1.0
3,COINFO,ALIEN & UNWIN,AUD,0.30,3.3,10000,0,AUS,1.0
4,COINFO,Amer Psych Ass.,AUD,0.33,3.3,10000,0,AUS,1.0


In [18]:
sup_pub.columns = map(str.lower, sup_pub.columns)
sup_pub = sup_pub.fillna('N/A')
for col in ['supplier','publisher', 'currency', 'shipmentorigin']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)
sup_pub.head()

,supplier,publisher,currency,discount,shipping cost,max quantity,min,shipmentorigin,shipping cost sea
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUS,1.0
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUS,1.0
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUS,1.0
3,COINFO,ALIEN & UNWIN,AUD,0.30,3.3,10000,0,AUS,1.0
4,COINFO,AMER PSYCH ASS.,AUD,0.33,3.3,10000,0,AUS,1.0


#### 4.4.1 Merge Supplier by Publisher with biblio

In [19]:
sup_pub_biblio = pd.merge(biblio_lp, sup_pub, how='inner', left_on = ['currency', 'publisher'], right_on = ['currency', 'publisher'])
sup_pub_biblio.head()

,isbn,currency,price,title,publisher,author,supplier,discount,shipping cost,max quantity,min,shipmentorigin,shipping cost sea
0,9780000002402,GBP,306.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0
1,9780000006493,GBP,772.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0
2,9780000009883,GBP,92.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0
3,9780000003201,GBP,265.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0
4,9780000079145,GBP,67.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0


In [20]:

sup_pub_biblio.rename(columns={'publisher': 'pub','shipping cost': 'shipping_cost', 'max quantity': 'max_qty', 'shipmentorigin': 'origin', 'shipping cost sea' : 'sea_shipping_cost'}, inplace = True)

sup_pub_biblio.head()

,isbn,currency,price,title,pub,author,supplier,discount,shipping_cost,max_qty,min,origin,sea_shipping_cost
0,9780000002402,GBP,306.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0
1,9780000006493,GBP,772.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0
2,9780000009883,GBP,92.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0
3,9780000003201,GBP,265.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0
4,9780000079145,GBP,67.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0


#### 4.5 Import short_disc and clean it

In [21]:
sp_disc = pd.read_excel("../../../../Vendors/. Short Discounted/all_short_discount_list.xlsx")
sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [22]:
sp_disc.columns = map(str.lower, sp_disc.columns)
sp_disc = sp_disc.fillna('N/A')
sp_disc['isbn'] = sp_disc['isbn'].astype(str)
for col in ['supplier', 'isbn']:
    sp_disc[col] = sp_disc.apply(lambda x: clean_up(x[col]), axis =1)
sp_disc.head()

,supplier,isbn,discount
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


#### 4.5.1 Merge sup_pub_biblio by sp_disc with biblio

In [23]:
sup_biblio_disc = pd.merge(sup_pub_biblio, sp_disc, how='left', on = ['supplier', 'isbn'])
sup_biblio_disc.head()

,isbn,currency,price,title,pub,author,supplier,discount_x,shipping_cost,max_qty,min,origin,sea_shipping_cost,discount_y
0,9780000002402,GBP,306.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN
1,9780000006493,GBP,772.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN
2,9780000009883,GBP,92.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN
3,9780000003201,GBP,265.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN
4,9780000079145,GBP,67.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN


In [24]:
sup_biblio_disc['discount'] = np.where(sup_biblio_disc['discount_y'].isna(), sup_biblio_disc['discount_x'],sup_biblio_disc['discount_y'])


#Drop unneccessary Columns

sup_biblio_disc.head()

,isbn,currency,price,title,pub,author,supplier,discount_x,shipping_cost,max_qty,min,origin,sea_shipping_cost,discount_y,discount
0,9780000002402,GBP,306.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN,0.55
1,9780000006493,GBP,772.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN,0.55
2,9780000009883,GBP,92.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN,0.55
3,9780000003201,GBP,265.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN,0.55
4,9780000079145,GBP,67.0,N/A,LWW,N/A,ABE,0.55,3.47,10000,0,UK,1.0,NaN,0.55


In [25]:
sup_biblio_disc = sup_biblio_disc[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost', 'sea_shipping_cost', 'max_qty', 'origin']]
sup_biblio_disc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin
0,9780000002402,N/A,LWW,N/A,ABE,GBP,306.0,0.55,3.47,1.0,10000,UK
1,9780000006493,N/A,LWW,N/A,ABE,GBP,772.0,0.55,3.47,1.0,10000,UK
2,9780000009883,N/A,LWW,N/A,ABE,GBP,92.0,0.55,3.47,1.0,10000,UK
3,9780000003201,N/A,LWW,N/A,ABE,GBP,265.0,0.55,3.47,1.0,10000,UK
4,9780000079145,N/A,LWW,N/A,ABE,GBP,67.0,0.55,3.47,1.0,10000,UK


### 4.6 Create table for WC

In [26]:
can_sup = biblio_lp[biblio_lp['currency'] == 'CND']
can_sup.columns = map(str.lower, can_sup.columns)
can_sup['supplier'] = 'WC'
can_sup['discount'] = np.where(can_sup['price']<150, 0.1, np.where((can_sup['price'] >=150) & (can_sup['price']<200),0.12,0.14))
can_sup['max_qty'] = 10000
can_sup['origin'] = 'CAN'
can_sup['shipping_cost'] = 2
can_sup.rename(columns={'publisher': 'pub'}, inplace = True)
can_sup['currency'] = 'CAD'
can_sup['sea_shipping_cost'] = 2

can_sup = can_sup[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost' , 'max_qty', 'origin']]

can_sup.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin
129,9780002007580,N/A,N/A,N/A,WC,CAD,22.95,0.1,2,2,10000,CAN
146,9780006393443,N/A,N/A,N/A,WC,CAD,7.99,0.1,2,2,10000,CAN
147,9780006393733,N/A,N/A,N/A,WC,CAD,19.95,0.1,2,2,10000,CAN
148,9780006393825,N/A,N/A,N/A,WC,CAD,19.95,0.1,2,2,10000,CAN
149,9780006393832,GILEAD,HARPER COLLINS,MARILYNNE ROBINSON,WC,CAD,18.95,0.1,2,2,10000,CAN


## 5. Append SupplierMegaLis, PublisherSuppliers and WesterCampus

In [27]:
all_sup = megalist_pub.append([sup_biblio_disc, can_sup], ignore_index= True)

all_sup_biblio = pd.merge(all_sup,biblio[['isbn', 'publisher']] , how='left', on = 'isbn')

all_sup_biblio['pub'] = np.where(all_sup_biblio['pub'].isna(), all_sup_biblio['publisher'], all_sup_biblio['pub'])

all_sup = all_sup_biblio[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost' , 'max_qty', 'origin']]
all_sup['sea_shipping_cost'] = np.where( (all_sup['origin'] == 'US') | (all_sup['origin'] == 'CAN') , all_sup['shipping_cost'], all_sup['sea_shipping_cost'])
all_sup.isna().sum()

isbn                   0
title                458
pub                  458
author               458
supplier               0
currency               0
price                  0
discount               0
shipping_cost          0
sea_shipping_cost      0
max_qty                0
origin                 0
dtype: int64

## 6. Add Fees

In [28]:
#Invoice Fee
all_sup['invoice_fee'] = np.where(     # IF
    all_sup['supplier'].str.contains('ALEK'), 0.005,   #Condition and True
    np.where(  # IF False
        (all_sup['supplier'].str.contains('LAURENTIU')) | (all_sup['supplier'].str.contains('SENAD')),0.02,   #Condition and True
        np.where(  #IF FALSE
            (all_sup['supplier'].str.contains('ARMANDO')) | (all_sup['supplier'].str.contains('FELIPE')),0.01, #Condition and True
            np.where(   #IF FALSE
                (all_sup['supplier'] == 'COINFO'), -1/11, #Condition and True
                0)   # FALSE
                )
                )
                )
# lp_fee

all_sup['lp_fee'] = np.where(
    (all_sup['supplier'].str.contains('BILLSON') & (all_sup['discount'] == 0 )), 0.03,0)

all_sup.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1.0,1000,UK,0.0,0.0
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1.0,1000,UK,0.0,0.0
2,9781861345806,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,28.00,0,3.47,1.0,150,AUS,0.0,0.0
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1.0,1000,UK,0.0,0.0
4,9781861345837,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,35.00,0,3.47,1.0,150,AUS,0.0,0.0


## 7. import and Merge exchage rate with all suppliers

In [29]:
exch_df = pd.read_sql(exchange_rate,proc_db)
exch_df.rename(columns={'name': 'currency'}, inplace = True)
all_sup_exc = pd.merge(all_sup, exch_df, how='left', on = 'currency')
all_sup_exc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1.0,1000,UK,0.0,0.0,1.245904
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1.0,1000,UK,0.0,0.0,1.245904
2,9781861345806,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,28.00,0,3.47,1.0,150,AUS,0.0,0.0,0.691870
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1.0,1000,UK,0.0,0.0,1.245904
4,9781861345837,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,35.00,0,3.47,1.0,150,AUS,0.0,0.0,0.691870


## 8. Cost Before exchange rate and landed cost

In [30]:
all_sup_exc['inv_price_before_exc'] = all_sup_exc['price']*(1-all_sup_exc['discount']+all_sup_exc['lp_fee']*(1+all_sup_exc['invoice_fee']))
all_sup_exc['regular_landed_cost'] = all_sup_exc['inv_price_before_exc'] * all_sup_exc['rate'] + all_sup_exc['shipping_cost']
all_sup_exc['sea_landed_cost'] = all_sup_exc['inv_price_before_exc'] * all_sup_exc['rate'] + all_sup_exc['sea_shipping_cost']
all_sup_exc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1.0,1000,UK,0.0,0.0,1.245904,3.0938,7.32458,4.85458
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1.0,1000,UK,0.0,0.0,1.245904,15.5,22.7815,20.3115
2,9781861345806,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,28.00,0,3.47,1.0,150,AUS,0.0,0.0,0.691870,28,22.8424,20.3724
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1.0,1000,UK,0.0,0.0,1.245904,23.529,32.7849,30.3149
4,9781861345837,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,35.00,0,3.47,1.0,150,AUS,0.0,0.0,0.691870,35,27.6855,25.2155


## 9. Restrictions

In [31]:
rest = pd.read_excel("../../../../Vendors/. Restrictions\All Restricted List.xlsx")
rest.columns = map(str.lower, rest.columns)
rest.rename(columns={'isbn13': 'isbn'}, inplace = True)
rest['isbn'] = rest['isbn'].astype(str)
for col in ['isbn', 'vendor']:
    rest[col] = rest.apply(lambda x: clean_up(x[col]), axis =1)

rest = rest[['isbn','vendor']]
rest.head()

,isbn,vendor
0,9780323567428,RISKY
1,9780500292105,RISKY
2,9781936523443,RISKY
3,9780134149530,RISKY
4,9780134205588,RISKY


In [32]:
# restricted to all suppliers
risky = rest[rest['vendor'] == 'RISKY']
all_sup_exc = all_sup_exc[~all_sup_exc['isbn'].isin(list(risky['isbn']))]

In [33]:
# restriction per supplier
all_rest = pd.merge(all_sup_exc, rest, how='left', left_on = ['isbn','supplier'], right_on = ['isbn', 'vendor'] )
all_rest[~(all_rest['vendor'].isna())].head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost,vendor
24611,9781260187014,PACKAGE: ELEMENTARY STATISTICS: A STEP BY STEP...,MCGRAW,ALLAN G. BLUMAN,RAJAN,USD,276.65,0.48,0.50,0.5,10000,US,0.000,0.0,1.000000,143.858,144.358,144.358,RAJAN
41779,9781284107913,N/A,J&B,N/A,RAJAN,USD,44.95,0.38,0.50,0.5,10000,US,0.000,0.0,1.000000,27.869,28.369,28.369,RAJAN
41849,9781284126143,N/A,J&B,N/A,RAJAN,USD,79.95,0.38,0.50,0.5,10000,US,0.000,0.0,1.000000,49.569,50.069,50.069,RAJAN
42779,9780077831066,MOSAIC LEVEL 2 LISTENING/SPEAKING STUDENT BOOK...,MCGRAW,JAMI HANREDDY; ELIZABETH WHALLEY,ALEK,GBP,42.65,0.5,3.47,1.0,10000,UK,0.005,0.0,1.245904,21.325,30.0389,27.5689,ALEK
59156,9781285769431,MILADY STANDARD COSMETOLOGY,CENGAGE,MILADY,SOHRAB,SGD,83.50,0,10.00,1.2,10000,SINGAPORE,0.000,0.0,0.717983,83.5,69.9516,61.1516,SOHRAB


## 10. Getting all posible orders for cost_analysis

In [34]:
all_rest.drop(
    all_rest[
        (all_rest['supplier'] == all_rest['vendor'])
        ].index, inplace = True
)
all_rest.drop(columns=['vendor'], inplace = True)
all_rest = all_rest.sort_values(by=['regular_landed_cost'])
all_rest.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
2888303,9780786820771,N/A,HACHETTE LIVE,N/A,ALEK,USD,0.01,0.59,0.5,0.5,1000,US,0.005,0.0,1.000000,0.0041,0.5041,0.5041
2310759,9780194705493,N/A,OXFORD UP ELT,N/A,ALEK,GBP,0.14,0.43,0.5,0.5,10000,US,0.005,0.0,1.245904,0.0798,0.599423,0.599423
3691196,9780078022159,DATABASE SYSTEM CONCEPTS,MCGRAW,"KORTH, HENRY F., SUDARSHAN, S., SILBERSCHATZ, ...",RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.13,0.63,0.63
2338862,9780143135241,N/A,PENGUIN ADULT HC/TR,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.8774,0.8774
2338866,9780143135265,N/A,PENGUIN ADULT HC/TR,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.8774,0.8774


In [35]:
all_rest = all_rest.fillna('N/A')
all_rest.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
2888303,9780786820771,N/A,HACHETTE LIVE,N/A,ALEK,USD,0.01,0.59,0.5,0.5,1000,US,0.005,0.0,1.000000,0.0041,0.504100,0.504100
2310759,9780194705493,N/A,OXFORD UP ELT,N/A,ALEK,GBP,0.14,0.43,0.5,0.5,10000,US,0.005,0.0,1.245904,0.0798,0.599423,0.599423
3691196,9780078022159,DATABASE SYSTEM CONCEPTS,MCGRAW,"KORTH, HENRY F., SUDARSHAN, S., SILBERSCHATZ, ...",RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.630000
2338862,9780143135241,N/A,PENGUIN ADULT HC/TR,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.877400,0.877400
2338866,9780143135265,N/A,PENGUIN ADULT HC/TR,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.877400,0.877400


## Extra: Getting suppliers from UK, CAN, US

In [36]:
first_filter = all_rest[
    (all_rest['origin'] == 'UK')
    | (all_rest['origin'] == 'US')
    | (all_rest['origin'] == 'CAN')
    ]

first_filter.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
2888303,9780786820771,N/A,HACHETTE LIVE,N/A,ALEK,USD,0.01,0.59,0.5,0.5,1000,US,0.005,0.0,1.000000,0.0041,0.504100,0.504100
2310759,9780194705493,N/A,OXFORD UP ELT,N/A,ALEK,GBP,0.14,0.43,0.5,0.5,10000,US,0.005,0.0,1.245904,0.0798,0.599423,0.599423
3691196,9780078022159,DATABASE SYSTEM CONCEPTS,MCGRAW,"KORTH, HENRY F., SUDARSHAN, S., SILBERSCHATZ, ...",RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.630000
2338862,9780143135241,N/A,PENGUIN ADULT HC/TR,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.877400,0.877400
2338866,9780143135265,N/A,PENGUIN ADULT HC/TR,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.877400,0.877400


In [37]:
isbn_list = ['9781564240071',
'9780808051763',
'9781260433111',
'9780134480435',
'9780134874371',
'9780134396026',
'9780133796773',
'9781319105990',
'9780134580999',
'9780134730363',
'9780134730684',
'9780135191798',
'9780134995991',
'9780134740607',
'9780134817125',
'9781454881360',
'9781259870446',
'9780134319650',
'9780135163078',
'9780134649290',
'9780134696515',
'9781454893509',
'9780135184233',
'9781118912652',
'9780134744476',
'9780135191767',
'9781319107376',
'9780134461991',
'9780134741086',
'9781337902571',
'9781337395250',
'9780357133491',
'9781337694193',
'9781319218331',
'9780134382593',
'9780134410968',
'9780134290553',
'9780134441184',
'9781640209718',
'9781683288220',
'9781634603256',
'9780134696461',
'9780134105598',
'9780131481930',
'9780134879482',
'9780135163825',
'9780134484143',
'9781118879108',
'9781618532312',
'9781618533128',
'9780134433042',
'9780134740218',
'9781449604455',
'9780134998459',
'9781305965799',
'9781337619455',
'9781305658004',
'9781319042578',
'9780134756967',
'9780134988696',
'9780134604657',
'9781259827440',
'9781640208520',
'9780357033821',
'9781337613040',
'9780134729534',
'9780134605197',
'9781319126698',
'9781634605847',
'9781683289845',
'9781634608152',
'9781640201859',
'9781642422504',
'9780135186275',
'9780134203690',
'9780134729220',
'9781683283058',
'9780314266057',
'9781634608190',
'9781683280064',
'9781634605908',
'9780784414248',
'9781609302276',
'9780134637433',
'9781319058135',
'9781618533111',
'9780132337175',
'9780134685762',
'9780135231678',
'9781683281412',
'9781683286530',
'9781683288329',
'9781609304508',
'9781642424867',
'9781640205918',
'9781634605298',
'9781683282136',
'9781640206106',
'9781683284239',
'9781628101515',
'9780134696522',
'9780134493756',
'9780134469966',
'9780134641348',
'9780134606545',
'9780134751979',
'9781634597470',
'9781634603225',
'9781609303334',
'9781609304225',
'9781337624657',
'9780134599717',
'9780314283726',
'9781337630702',
'9781683282143',
'9781640202498',
'9780134878119',
'9780134995588',
'9780134302386',
'9780134758992',
'9780134555805',
'9781522105442',
'9781634605953',
'9781319039448',
'9781634608855',
'9781634605311',
'9780134526690',
'9780134891521',
'9780134898674',
'9780135116159',
'9780134641621',
'9780134220130',
'9780135183885',
'9780134860244',
'9780134689623',
'9780134494074',
'9781531008727',
'9780132938310',
'9780134320533',
'9780134670959',
'9780134597119',
'9780135057490',
'9780190618353',
'9781337560573',
'9780190698614',
'9781305951174',
'9780321986245',
'9781319143633',
'9781454896289',
'9780134609034',
'9780134394190',
'9780195124446',
'9781319104191',
'9781319104177',
'9780134602820',
'9780135435199',
'9780134543536',
'9780134479279',
'9781319050740',
'9781683289883',
'9780190847609',
'9781531009182',
'9781683282365',
'9781138218154',
'9781319065669',
'9781337794213',
'9781337553278',
'9781337408974',
'9781531009403',
'9781531005474',
'9781454873433',
'9780357033852',
'9780136008064',
'9780134606989',
'9781337798839',
'9780134710679',
'9780997117141',
'9780357117811',
'9780134392790',
'9780134016344',
'9780134060354',
'9781138211698',
'9780134219929',
'9780134238739',
'9781305494695',
'9781337798310',
'9780357026434',
'9780393667691',
'9780134740225',
'9781138036338',
'9781531001438',
'9781138103962',
'9780357042502',
'9781635500615',
'9780134031255',
'9781337900348',
'9780357370391',
'9781605358222',
'9780357370384',
'9780134695815',
'9780134041674',
'9780134696485',
'9781454896210',
'9781260153149',
'9780134168975',
'9780134181028',
'9780134380926',
'9780134039558',
'9780134701271',
'9780974934532',
'9780357026397',
'9780134875460',
'9781337696609',
'9781138292406',
'9780190625276',
'9781285851150',
'9781138720923',
'9781337567916',
'9780135169377',
'9780393624045',
'9781319060619',
'9780134549163',
'9780133976892',
'9780199343386',
'9781605356419',
'9780134784441',
'9780134802213',
'9780134678863',
'9780134857770',
'9781454897958',
'9781640209701',
'9781454856092',
'9781284153071',
'9781618532640',
'9780134709857',
'9780815366546',
'9780393284133',
'9780134300788',
'9780134484488',
'9780134286303',
'9780134484204',
'9780134477367',
'9780134714882',
'9780393640366',
'9781506339764',
'9781605357300',
'9781577666936',
'9781449627904',
'9780134808246',
]

In [38]:
uk_us_can = first_filter[first_filter['isbn'].isin(isbn_list)]
uk_us_can.to_csv('retail/uk_us_can.csv', index= False)
# all_suppliers = all_rest[all_rest['isbn'].isin(isbn_list)]
# all_suppliers.to_csv('retail/all_suppliers.csv', index= False)
# elsevier = all_rest[all_rest['pub'].str.contains('ELSEVIER')]
# elsevier.to_csv('retail/elsevier.csv', index= False)
elsevier_cengage = all_rest[all_rest['pub'].str.contains('CENG') | all_rest['pub'].str.contains('ELSE')]
# elsevier_cengage.drop_duplicates(subset ="isbn", keep = False, inplace = True) 
elsevier_cengage.to_csv('retail/elsevier_cengage.csv', index= False)


In [39]:
# all_rest.to_csv('important_files/python/ca_results.csv', index= False)


